In [1]:
import pandas as pd
import numpy as np
import random
import datetime

https://leetcode.com/problems/average-selling-price/?lang=pythondata

In [57]:
data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30], [3, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [58]:
prices

,product_id,start_date,end_date,price
0,1,2019-02-17,2019-02-28,5
1,1,2019-03-01,2019-03-22,20
2,2,2019-02-01,2019-02-20,15
3,2,2019-02-21,2019-03-31,30
4,3,2019-02-21,2019-03-31,30


In [59]:
units_sold

,product_id,purchase_date,units
0,1,2019-02-25,100
1,1,2019-03-01,15
2,2,2019-02-10,200
3,2,2019-03-22,30


**pandas.DataFrame.assign(kwargs)** : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html

* DataFrame에 새로운 column을 붙여주는(assign) 함수. 새로운 column이 붙은 DataFrame을 반환한다. 혹은 기존의 column값들을 수정하기 위해 사용할 수도 있다. 어쨌든 column에 변경사항을 주거나 새로운 column을 추가할 때 쓰는 함수.
* ***kwargs***에다가 원하는 column을 설정한다. dict형태로 넣어줘도 되고, callable(쉽게 말하면 함수?)을 집어넣으면 해당 연산을 수행한 결과가 column 값들로 설정된다.


In [60]:
"""
주어진 데이터에서 각 제품 별 판매금액의 평균을 구하는 문제.
"""

merged_df = prices.merge(units_sold, on='product_id')
condition = (merged_df['purchase_date'] >= merged_df['start_date']) & (merged_df['purchase_date'] <= merged_df['end_date'])
# loc를 사용할 때 이런 식으로 조건을 넣을 수도 있다.
filtered_df = merged_df.loc[condition, ['product_id','price','units']]
filtered_df

,product_id,price,units
0,1,5,100
3,1,20,15
4,2,15,200
7,2,30,30


In [61]:
# assign함수를 이용했다. revenue라는 새로운 column label을 만들어주고, 해당 column에 각각의 행에 있는 price와 units값들을 곱해 revenue값을 구해준다.
# assign수행 결과 반환되는 값을 기존의 filtered_df에다가 덮어씌운다. 안 그래도 되지만 변수를 덜 만드는 게 효율적(?)일 수도 있으니까 일단 해본다.
filtered_df = filtered_df.assign(revenue=filtered_df['price']*filtered_df['units'])
filtered_df

,product_id,price,units,revenue
0,1,5,100,500
3,1,20,15,300
4,2,15,200,3000
7,2,30,30,900


In [62]:
# 평균을 구하기 위해 revenue의 합과 units의 합을 구하고, 이제 grouped_df에 새 column label을 만들고 거기에 그 값들을 넣는다.
grouped_df = filtered_df.groupby('product_id').agg({'revenue':'sum','units':'sum'}).reset_index() # reset_index는 groupby의 as_index를 False로 두는 것과 같은 효과
grouped_df = grouped_df.assign(average_price = (grouped_df['revenue'] / grouped_df['units']).round(2)) # grouped_df["average_price"] = (grouped_df["revenue"] / grouped_df["units"]).round(2)
grouped_df = grouped_df[['product_id','average_price']]

In [63]:
prices['product_id'].nunique()

3

In [68]:
prices['product_id'].nunique() != grouped_df['product_id'].nunique()

True

In [69]:
zero_avg = [id for id in prices['product_id'].values if id not in grouped_df['product_id'].values]
zero_avg

[3]

In [70]:
# 근데 풀다보니 아예 안 팔린 제품이 있을 때는 안 팔린 제품은 결과 goupred_df에 포함이 안 된다. 이게 안 팔리면 0이라고 표시를 해야 한다.
if prices['product_id'].nunique() != grouped_df['product_id'].nunique():
  zero_avg = [id for id in prices['product_id'].values if id not in grouped_df['product_id'].values]
  zero_avg_row = pd.DataFrame({'product_id':zero_avg, 'average_price':0})
  final_grouped_df = pd.concat([grouped_df, zero_avg_row], ignore_index=True)
final_grouped_df

,product_id,average_price
0,1,6.96
1,2,16.96
2,3,0.0


In [71]:
def average_selling_price(prices: pd.DataFrame, units_sold: pd.DataFrame) -> pd.DataFrame:
    merged_df = prices.merge(units_sold, on='product_id')
    condition = (merged_df['purchase_date'] >= merged_df['start_date']) & (merged_df['purchase_date'] <= merged_df['end_date'])
    filtered_df = merged_df.loc[condition, ['product_id','price','units']]
    filtered_df = filtered_df.assign(revenue = filtered_df['price']*filtered_df['units'])

    grouped_df = filtered_df.groupby('product_id').agg({'revenue':'sum', 'units':'sum'}).reset_index()
    grouped_df = grouped_df.assign(average_price = round(grouped_df['revenue']/grouped_df['units'], 2))

    if prices['product_id'].nunique() != grouped_df['product_id'].nunique():
        zero_avg = [id for id in prices['product_id'].values if id not in grouped_df['product_id'].values]
        zero_avg_row = pd.DataFrame({'product_id':zero_avg, 'average_price':0})
        final_grouped_df = pd.concat([grouped_df, zero_avg_row], ignore_index=True)
        return final_grouped_df[['product_id','average_price']]
    return grouped_df[['product_id','average_price']]

In [72]:
df = prices.merge(units_sold, on='product_id')
df

,product_id,start_date,end_date,price,purchase_date,units
0,1,2019-02-17,2019-02-28,5,2019-02-25,100
1,1,2019-02-17,2019-02-28,5,2019-03-01,15
2,1,2019-03-01,2019-03-22,20,2019-02-25,100
3,1,2019-03-01,2019-03-22,20,2019-03-01,15
4,2,2019-02-01,2019-02-20,15,2019-02-10,200
5,2,2019-02-01,2019-02-20,15,2019-03-22,30
6,2,2019-02-21,2019-03-31,30,2019-02-10,200
7,2,2019-02-21,2019-03-31,30,2019-03-22,30


In [73]:
df.groupby('product_id').agg({'price':'mean', 'units':'mean', 'start_date':['min','max']})

price  units start_date           
            mean   mean        min        max
product_id                                   
1           12.5   57.5 2019-02-17 2019-03-01
2           22.5  115.0 2019-02-01 2019-02-21

In [75]:
df.loc[df['start_date']=='2019-02-17', ['price','purchase_date', 'start_date']]

,price,purchase_date,start_date
0,5,2019-02-25,2019-02-17
1,5,2019-03-01,2019-02-17
